Taking a binary mask from our model the follwoing code will produce the area and azimuth angle for each building footprint.

In [102]:
%config Completer.use_jedi = False

In [16]:
from solaris.data import data_dir
import solaris as sol
import os
import skimage
import geopandas as gpd
from matplotlib import pyplot as plt
from shapely.ops import cascaded_union
import cv2
import rasterio as rio
import pycocotools.mask as mask_util
import shapely
import math

In [20]:
class Post_Process:
    def __init__(self, bin_mask , tif_img, crs= 'EPSG:5703'):
        '''
        class takes model prediction (binary mask array) 
        and reference image, for post processing
        '''
        self.bin_mask = bin_mask
        self.tif_img = tif_img
        self.geo_df = None
        self.crs = crs
        
    
    def __get_geo_df(self, output_path=None):
        '''
        Takes a tif image and returns a geo dataframe
        '''
        if output_path:
            sol.vector.mask.mask_to_poly_geojson(pred_arr=self.bin_mask, 
                                                 reference_im=self.tif_img, 
                                                 output_path=output_path,
                                                 output_type='geojson', simplify=True)
            
            self.geo_df= gpd.read_file(output_path)
            # Update projection 
            self.geo_df = self.geo_df.to_crs(self.crs) # in meter

        # datagrame was not saved    
        else:
            geoms = sol.vector.mask.mask_to_poly_geojson(pred_arr=self.bin_mask, 
                                                 reference_im=self.tif_img,  
                                                 output_type='geojson', simplify=True)
            self.geo_df = gpd.GeoDataFrame(geoms, crs=self.crs)     
        
        return self.geo_df

        
    
    def __calc_azimuth(self, g):
        '''takes a geometry  and returns the angle'''        
        a = g.minimum_rotated_rectangle
        l = a.boundary
        coords = [c for c in l.coords]
        segments = [shapely.geometry.LineString([a, b]) for a, b in zip(coords,coords[1:])]
        longest_segment = max(segments, key=lambda x: x.length)

        p1, p2 = [c for c in longest_segment.coords]
        angle = math.degrees(math.atan2(p2[1]-p1[1], p2[0]-p1[0]))
        return angle
    
    def __get_area(self):
        '''
        calculate area for our polygons and append to the geo_df
        '''
        # If it we have the df
        # Get the area and append to the df
        if self.geo_df is not None :
            # Calculate the area
            self.geo_df['area(square feet)'] = ((self.geo_df['geometry'].area)/10.764) # In sequare feet
        else:
            print("No dataframe acquired, yet. Use get_geo_df")

        return self.geo_df
    
    
    def __get_azimuth(self):
        # Make sure we already have the geo_df calculated
        list_azimuth = []
        if self.geo_df is not None :
            for i in range(len(self.geo_df)):
                g = self.geo_df.iloc[i].geometry
                angle = self.__calc_azimuth(g)
                list_azimuth.append(angle)
                
            self.geo_df['angle'] = list_azimuth
        
        else: 
             print("No dataframe acquired, yet. Use get_geo_df")
        
        return self.geo_df
    
    def get_full_post_process(self):
        '''Full post process mask'''
        self.geo_df = self.__get_geo_df()
        self.geo_df = self.__get_area()
        self.geo_df = self.__get_azimuth()
        return  self.geo_df
        
        

In [21]:
# Test our pipeline
image = skimage.io.imread(os.path.join(data_dir, 'sample_geotiff.tif'))
mask = cv2.imread('D:/mask.jpg', 0)

In [22]:
obj=Post_Process(mask, image)
geo_df_full = obj.get_full_post_process()

geo_df_full

,geometry,value,area(square feet),angle
0,"POLYGON ((644.000 0.000, 644.000 1.000, 645.00...",255.0,15.328874,165.963757
1,"POLYGON ((459.000 0.000, 459.000 2.000, 458.00...",255.0,67.911557,116.565051
2,"POLYGON ((816.000 1.000, 816.000 2.000, 795.00...",255.0,89.929394,90.000000
3,"POLYGON ((0.000 3.000, 0.000 54.000, 2.000 54....",255.0,56.577480,90.000000
4,"POLYGON ((59.000 67.000, 59.000 69.000, 58.000...",255.0,77.294686,26.565051
5,"POLYGON ((434.000 44.000, 434.000 48.000, 433....",255.0,105.815682,98.972627
6,"POLYGON ((813.000 58.000, 813.000 59.000, 806....",255.0,88.628763,90.000000
7,"POLYGON ((571.000 78.000, 571.000 79.000, 568....",255.0,93.366778,-48.366461
8,"POLYGON ((228.000 118.000, 228.000 119.000, 22...",255.0,6.874768,-168.690068
9,"POLYGON ((133.000 98.000, 133.000 100.000, 132...",255.0,84.262356,45.000000


### Reference
- https://epsg.io/
- https://kb.orbitgt.com/209/technology/core/crs
- https://gis.stackexchange.com/questions/218450/getting-polygon-areas-using-geopandas